In [1]:
import numpy as np
import nltk
import os, glob
import pandas as pd
import copy
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords

In [2]:
def clean(text):
    return unicode(''.join([i if ord(i) < 128 else ' ' for i in text]))

In [3]:
import spacy
from spacy.lang.en import English
nlp = spacy.load('en_core_web_sm')

In [4]:
def preprocessing(text):
    doc = clean(text)

    doc = str(doc).lower()
    doc = nlp(clean(doc))
    
    # we add some words to the stop word list
    texts, article = '', []
    for w in doc:
        if str(w) != '\n':
            texts = texts + ' ' + str(w.text)


    i = 0
    while i < len(texts):
        if((ord(texts[i])>=65 and ord(texts[i]) <=90) or ord(texts[i]) == 46):
            pass
        elif((ord(texts[i])>=97 and ord(texts[i]) <=122) or ord(texts[i]) ==95 or ord(texts[i]) == 32):
            pass
        else:
            texts = texts.replace(texts[i], '')
        i +=1
            #texts.remove(texts[i])
            
    return texts


def noun_adj(texts):

    tokens = nltk.sent_tokenize(texts)
    #print len(tokens)
    keyphrases = []
    for i in range(len(tokens)):
        tok = nltk.word_tokenize(tokens[i])
        #print tok
        pos = nltk.pos_tag(tok)
        #print pos
        #print pos
        l, ll = '', []
        for word, pos in pos:
            if (pos == 'NN' or pos == 'NNP' or pos == 'NNS' or pos == 'NNPS' or pos == 'JJ' or pos == 'JJR' or pos == 'JJS'):
                if len(l) != 0:
                    l = l + " " + word
                else:
                    l = word
            elif len(l) != 0:
                ll.append(l)
                l = ''
        if len(l) != 0:
            ll.append(l)
            l = ''
        keyphrases.append(ll)
        ll = []

    return keyphrases

def mainKeyphrases(keyphrases):
        #not w.is_stop and not w.is_punct and not w.like_num
    #print ll
    #print nltk.word_tokenize(ll[9])
    mainKeys, ke, kk = [], [], []
    #for i in range(len(keyphrases)):
    i=0
    while i < len(keyphrases):
        #############
        if len(keyphrases[i]) == 0:
            del keyphrases[i]
            i -= 1
        else:
            for j in range(len(keyphrases[i])):
                    #print i,j
                    #print keyphrases[i][j]
                kk =  nltk.word_tokenize(keyphrases[i][j])
                    #print kk
                k=0
                while k < len(kk):
                    ####################3
                    if len(kk[k]) < 3:
                        del kk[k]
                        k -= 1
                    k+=1
                            
                ke.append(kk)
            mainKeys.extend(ke)
            ke = []
        i += 1
    #print mainKeys
    
    #joining keyphrases
    keyphrase = []
    for i in mainKeys:
        keyphrase.append(' '.join(i))
        
    #removing empty items
    i=0
    while i < len(keyphrase):
        if len(keyphrase[i]) < 1:
            keyphrase.remove(keyphrase[i])
            i-=1
        i+=1
    
    return keyphrase

In [5]:
def dataframe():
    #co-occur window size 2
    for ind, i in enumerate(filtered[k]):
        for index, j in enumerate(filtered[k]):
            if i != j:
                ii = ind - index
                if ii < 2 and ii > -2:
                    df1[i][j] = df1[i][j] + 1
                    
def vector_creation():
    for i in range(0, len(df1)):
        vector.append(1/float(len(df1)))
    return vector

def vector_update(vector):
    vector = df1.dot(vector)
    #np.savetxt('vector.txt', vector, fmt='%f')
    return vector
        
def normalization():
    for i, row in enumerate(df1.values):
        summ= 0
        for j, cols in enumerate(df1.values):
            summ = summ + df1[df1.index[i]][df1.index[j]]

        for j, cols in enumerate(df1.values):
            df1[df1.index[i]][df1.index[j]] = df1[df1.index[i]][df1.index[j]] / summ
            
def FinalTextRank(vector):
    a = 0.85
    i = 0
    a_sum = 0
    #print vector
    for df_row in df1:
        #print df1[df_row][0]
        a_sum = 0
        for df_col in df1.columns:
            summ = 0
            if df1[df_row][df_col] != 0:
                for dd in range(0, len(df1)):
                    summ = summ + df1[df_col][dd]
                a_sum = a_sum + np.multiply(df1[df_row][df_col] / summ, vector[df_col])    
        a_sum = np.multiply(a, a_sum) 
        vector[i] = np.multiply((1-a), vector[i])
        vector[i] = vector[i] + a_sum
        i = i + 1
        
    return vector

In [12]:
filtered = []
duplicates = []
final_vector = []
for i in range(1,2):
    lee_train_file = "art_and_culture.txt"
    text = open(lee_train_file).read()
    texts = preprocessing(text)
    keyphrases = noun_adj(texts)
    filtered.append(mainKeyphrases(keyphrases))
    duplicates.append(list(set(filtered[len(filtered)-1])))


for k in range(0,len(duplicates)):
    pd.options.display.float_format = '{:,.2f}'.format
    df1 = pd.DataFrame(0, index= duplicates[k], columns=duplicates[k])
    df1 = df1.astype(float)
    ll = df1.index.values.tolist()
    #final_list.append(ll)
    vector = []
    position = []

    dataframe()
    vector = vector_creation()
    normalization()
    vector = vector_update(vector)
    vector = FinalTextRank(vector)      
    final_vector.append(vector)

In [13]:
#top most keyphrases
predicted_keyphrases = []
predicted = []
vect = copy.deepcopy(final_vector)
for i in range(0, len(vect)):
    try:
        for j in range(0, 10):
            predicted.append(vect[i].idxmax())
            vect[i] = vect[i].drop(vect[i].idxmax())
    except:
        b = 0
    predicted_keyphrases.append(predicted)
    predicted = []

In [14]:
predicted_keyphrases

[['north america top grossing concert entertainers',
  'crowd',
  'tennis player juan monaco',
  'thursday march',
  'beautiful',
  'mansion',
  'traditional pop vocal album',
  'purple orchids',
  'buenos aires',
  'media']]